<a href="https://colab.research.google.com/github/TheReallyBigApple/CunyAssignments/blob/main/DATA607/Data607_Week2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data607_Week2

Tom Buonora
September 4, 2021

<br><br><br><br>

<center><font size="7" color="purple">Movie Ratings : Friends vs. IMDB</font></center>

<br><br><br><br><br>

This exercise queries a small dataset from MySQL to look at how 5 friends rated 6 movies.
The question is how do our friends ratings compare to IMDB.
The schema is normalized across 3 tables as follow.

<br><br><br><br>

Click on the links to see the files:

<br><br>

<table>
  <tr>
    <th>Description</th>
    <th>DB Table</th>
    <th>CSV File</th>
  </tr>
  <tr>
    <td>607_Movies.csv</td>
    <td>movies</td>
    <td><a href="https://github.com/TheReallyBigApple/CunyAssignments/blob/main/DATA607/607_Movies.csv"> 6 movies</td>
  </tr>
    <tr>
        <td>5 Friends</td>
        <td>people</td>
      <td><a href="https://github.com/TheReallyBigApple/CunyAssignments/blob/main/DATA607/607_People.csv">607_People.csv</td>
    </tr>
     <tr>
         <td>Movie,Person, Rating</td>
         <td>ratings</td>
     <td><a href="https://github.com/TheReallyBigApple/CunyAssignments/blob/main/DATA607/607_Ratings.csv">607_Ratings.csv</td>
    </tr>
</table>

<br><br><br><br>
All Database code is [here](https://github.com/TheReallyBigApple/CunyAssignments/blob/main/DATA607/607_week2.sql)
<br><br><br><br>


In [ ]:
import os
import mysql.connector
from mysql.connector import Error
from IPython.display import display, HTML,display_html 
import pandas as pd

<br><br><br><br>

The ratings file has 30 entries but some of my friends havent seen the movie yet so they have a blank rating.
<br>
We will handle those in the R code.
<br>

1) Get password from the environment
<br>


In [ ]:
KEY='DATA607_PASSWORD'
mysql_password = os.getenv(KEY)
# to set password, as administrator, setx -m  DATA607_PASSWORD <password>

<br>
2) Connect to database
<br>

In [ ]:

try:
        conn = mysql.connector.connect(host='localhost',
                                       database='DATA607',
                                       user='root',
                                       password=mysql_password)
        if conn.is_connected():
            print('Connected to MySQL database')


        cursor = conn.cursor()
      

except Error as e:
        print("Failed to connect to database " , e)
        
   


Connected to MySQL database


<br>
3) Query each table into data frames.
<br>

In [ ]:
try:
    cursor.execute("select * from movies")
    movies_df = pd.DataFrame(cursor.fetchall())
except Error as e:
    print("Failed to select from movies " , e)

try:
    cursor.execute("select * from people")
    people_df = pd.DataFrame(cursor.fetchall())
except Error as e:
    print("Failed to select from people " , e)

try:
    cursor.execute("select * from ratings")
    ratings_df = pd.DataFrame(cursor.fetchall())
except Error as e:
    print("Failed to select from ratings " , e)
    

conn.close()

<br>
4) Restore the column headers
<br>

In [ ]:
movies_df.columns = ['id', 'name', 'imdb_rating', 'lead_actor']
people_df.columns = ['id', 'name']
ratings_df.columns = ['people_id', 'movie_id', 'rating']

<br>
5) Remove the carriage returns
<br>

In [ ]:
movies_df['lead_actor'] = movies_df['lead_actor'].str.strip()
people_df['name'] = people_df['name'].str.strip()

<br>
6) Display tables
<br>

In [ ]:
df1_styler = movies_df.head(n=3).style.set_table_attributes("style='display:inline'").set_caption('Movies')
df2_styler = people_df.head(n=3).style.set_table_attributes("style='display:inline'").set_caption('People')

space = "\xa0" * 50
display_html(df1_styler._repr_html_()+space+df2_styler._repr_html_(), raw=True)

Movies id name imdb_rating lead_actor 
 
 0 
 1 
 I Care a lot 
 6.300000 
 Rosamund Pike 
 
 
 1 
 2 
 State of Play 
 7.100000 
 Ben Affleck 
 
 
 2 
 3 
 The Interpreter 
 6.400000 
 Nicole Kidman 
 
                                                    People id name 
 
 0 
 1 
 Tom 
 
 
 1 
 2 
 Vera 
 
 
 2 
 3 
 Mom

In [ ]:
ratings_df.head(n=5).style \
   .set_caption('Ratings') \
  .highlight_null('lightgray')

,people_id,movie_id,rating
0,1,1,4.000000
1,1,2,3.000000
2,1,3,5.000000
3,1,4,nan
4,1,5,nan


<br>
6) Remove the null records
<br>

In [ ]:
ratings_df=ratings_df.dropna(subset=['rating'])

<br>
7) Merge the 3 dataframes into one
<br>

In [ ]:
merged_df = pd.merge(movie_df,ratings_df, left_on=['id'],right_on=['movie_id'])
merged_df = pd.merge(people_df,merged_df, left_on=['id'],right_on=['people_id'])
merged_df = merged_df[["name_x", "name_y","imdb_rating", "rating"]]
merged_df.columns = ['Friend', 'Movie', 'imdb_rating', 'rating']

<br>
8) Now answer the question.
<br>

In [ ]:

agg_func_count = {
        'rating': ['count', 'mean'],
        'imdb_rating':['first']
        }
answer_df = merged_df.groupby(['Movie']).agg(agg_func_count)
answer_df.columns = ['n_ratings', 'avg rating', 'imdb rating']


                                       

In [ ]:
answer_df

,n_ratings,avg rating,imdb rating
Movie,,,
I Care a lot,5,3.400000,6.3
Respect,3,3.333333,6.6
Selena,4,2.750000,6.3
State of Play,4,3.750000,7.1
The Farewell,4,4.000000,7.6
The Interpreter,2,4.500000,6.4



The same exercise was done in R [here](https://github.com/TheReallyBigApple/CunyAssignments/blob/main/DATA607/Data607_Week2.html)